In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from scipy.spatial import cKDTree
import pickle

# Predefined list of 10 selectable places
es_data = [
    ("Sri Venkateswara Temple", "Prestigious Temple", "Temple", 13.6839, 79.3475),
    ("Sri Padmavathi Temple", "Prestigious Temple", "Temple", 13.6288, 79.4322),
    ("Kapila Theertham", "Small Temple", "Temple", 13.6206, 79.3905),
    ("Govindaraja Swamy Temple", "Prestigious Temple", "Temple", 13.6350, 79.4186),
    ("Sri Kalahasti Temple", "Prestigious Temple", "Temple", 13.7496, 79.7037),
    ("Kanipakam Temple", "Prestigious Temple", "Temple", 13.2047, 79.2671),
    ("Sri Venkateswara Zoological Park", "Zoo", "Park", 13.7545, 79.4832),
    ("Sri Vari Museum", "Museum", "Museum", 13.6835, 79.3472),
    ("Alivelu Mangapuram", "Prestigious Temple", "Temple", 13.6043, 79.4698),
    ("Srinivasa Mangapuram", "Prestigious Temple", "Temple", 13.6013, 79.4924)
]

places_df = pd.DataFrame(es_data, columns=["Place Name", "God/Goddess/Attraction", "Category", "Latitude", "Longitude"])

# Load full dataset of 100 places
file_path = "C:/Users/TRIVENI/Downloads/Tirumala_Tourist_Places_53.xlsx"
df = pd.read_excel(file_path)

required_columns = {"Place Name", "God/Goddess/Attraction", "Category", "Latitude", "Longitude"}
if not required_columns.issubset(df.columns):
    raise ValueError("Excel file is missing required columns")

# Precompute and cache nearest neighbors using cKDTree
cache_file = "nearest_neighbors.pkl"
try:
    with open(cache_file, "rb") as f:
        nearest_neighbors = pickle.load(f)
except FileNotFoundError:
    nearest_neighbors = {}
    coords = df[["Latitude", "Longitude"]].values
    tree = cKDTree(coords)  # Create the KDTree from the full dataset

    for index, row in places_df.iterrows():
        base_location = (row["Latitude"], row["Longitude"])
        radius = 5 / 111  # Max distance in degrees (approx. 1° ≈ 111 km)

        # Query nearest places within the radius
        indices = tree.query_ball_point(base_location, r=radius)
        nearby_places = df.iloc[indices].copy()
        
        # Calculate accurate geodesic distances
        nearby_places["Distance (km)"] = [
            geodesic(base_location, (lat, lon)).km for lat, lon in nearby_places[["Latitude", "Longitude"]].values
        ]
        
        nearest_neighbors[row["Place Name"]] = nearby_places

    with open(cache_file, "wb") as f:
        pickle.dump(nearest_neighbors, f)

# Get user input
print("Select a base location from the following:")
for idx, row in places_df.iterrows():
    print(f"{idx + 1}. {row['Place Name']}")

base_index = int(input("Enter the number corresponding to your location: ")) - 1
if base_index < 0 or base_index >= len(places_df):
    print("Invalid selection. Exiting.")
    exit()

base_location = places_df.loc[base_index, "Place Name"]
max_distance = float(input("Enter the maximum distance (in km): "))

# Retrieve precomputed nearest places
nearby_places = nearest_neighbors.get(base_location, pd.DataFrame())
nearby_places = nearby_places[nearby_places["Distance (km)"] <= max_distance]
nearby_places["Rank"] = range(1, len(nearby_places) + 1)

# Display results
if not nearby_places.empty:
    print(nearby_places[["Rank", "Place Name", "God/Goddess/Attraction", "Category", "Distance (km)"]].to_string(index=False))
else:
    print("No places found within the given range.")


Select a base location from the following:
1. Sri Venkateswara Temple
2. Sri Padmavathi Temple
3. Kapila Theertham
4. Govindaraja Swamy Temple
5. Sri Kalahasti Temple
6. Kanipakam Temple
7. Sri Venkateswara Zoological Park
8. Sri Vari Museum
9. Alivelu Mangapuram
10. Srinivasa Mangapuram
Enter the number corresponding to your location: 1
Enter the maximum distance (in km): 50
 Rank                                                     Place Name    God/Goddess/Attraction           Category  Distance (km)
    1                              Sri Venkateswara Temple, Tirumala         Lord Venkateswara         Devotional       0.000608
    2                         Sri Govindaraja Swamy Temple, Tirupati          Lord Govindaraja         Devotional       0.002571
    3                                Akasa Ganga Theertham, Tirumala            Holy Waterfall          Holy Site       0.002571
    4                                     Chakra Theertham, Tirumala           Holy Water Body          H